In [2]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np
# Store excel file locations to variables

Com_2012 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\5. S2P1.3 Governance\2012_com_s5-q14to22.csv"
Com_2013 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\5. S2P1.3 Governance\2013_s2_q13q24_focus.csv"
Com_2014 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\5. S2P1.3 Governance\2014_s2_q2q16_community.csv"

# Read excel files 
df_2012 = pd.read_csv(Com_2012)
df_2013 = pd.read_csv(Com_2013)
df_2014 = pd.read_csv(Com_2014)



In [5]:
#This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.

# Rename columns in df to df_2012 column names
df_2012.rename(columns={
    'program': 'program_id',
    'S5Q6': 's2_q8',
    'S5Q7': 's2_q9',
    'S5Q11': 's2_q11',
    'S5Q8': 'G_Prgm_ILoM',
    'S5Q9_1': 'G_Prgm_Purp1',
    'S5Q9_2': 'G_Prgm_Purp2',
    'S5Q9_3': 'G_Prgm_Purp3',
    'S5Q10': 'G_Prgm_Decide',
    'S5Q12': 'G_PrgmEffC_5y',
    'S5Q13': 'G_Prgm_2007NoHHB',
    'round': 'Survey_Round',
    'community': 'Community',
    'fid': 'FID',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)

# Rename columns in df to df_2013 column names
df_2013.rename(columns={
    'prog_id': 'program_id',
    'program_name': 'program_name',
    's2_q6': 's2_q8',
    's2_q7': 's2_q9',
    's2_q8': 'G_Prgm_ILoM',
    's2_q9_first': 'G_Prgm_Purp1',
    's2_q9_second': 'G_Prgm_Purp2',
    's2_q9_third': 'G_Prgm_Purp3',
    's2_q10': 'G_Prgm_Decide',
    's2_q12': 'G_PrgmEffC_1y'
}, inplace=True)


# df_2014 doesn't need renaming as it is the reference

In [11]:
# Define the column mappings based on the provided positions
# Make dictionaries for each year with updated names
# replace with 'None' where there are no columns
#Here are the updated mapping lists for the given datasets:


mapping_2012 = [
    'cid',
    'program_id',
    None,
    's2_q8',
    's2_q9',
    None,
    's2_q11',
    'G_Prgm_ILoM',
    'G_Prgm_Purp1',
    'G_Prgm_Purp2',
    'G_Prgm_Purp3',
    'G_Prgm_Decide',
    None,
    'G_PrgmEffC_5y',
    'G_Prgm_2007NoHHB',
    'Survey_Round',
    'Community',
    'FID',
    'P_ID',
    'D_ID',
    'T_ID',
    'UC_ID',
    'M_ID'
]

mapping_2013 = [
     'cid',
    'program_id',
    'program_name',
    's2_q8',
    's2_q9',
    None,
    's2_q11',
    'G_Prgm_ILoM',
    'G_Prgm_Purp1',
    'G_Prgm_Purp2',
    'G_Prgm_Purp3',
    'G_Prgm_Decide',
    'G_PrgmEffC_1y',
    None, None, None, None, None, None, None, None, None, None
]


mapping_2014 = [
    'cid',
    'program_id',
    'program_name',
    's2_q8',
    's2_q9',
    's2_q10',
    's2_q11',
    None, None, None, None, None, None, None, None, None, None, None, None,
    None, None, None, None
]





In [12]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [13]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [14]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0', 'Survey_Round', 'cid', 'Community', 'prog_id', 'prog', 'S5Q14', 'S5Q15', 'S5Q16', 'S5Q17', 'S5Q18', 'S5Q19', 'S5Q20', 'S5Q21A', 'S5Q21B', 'S5Q21C', 'S5Q21D', 'S5Q22A', 'S5Q22B', 'FID', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column cid
Column program_id not found in DataFrame. Adding NaNs.
Column s2_q8 not found in DataFrame. Adding NaNs.
Column s2_q9 not found in DataFrame. Adding NaNs.
Column s2_q11 not found in DataFrame. Adding NaNs.
Column G_Prgm_ILoM not found in DataFrame. Adding NaNs.
Column G_Prgm_Purp1 not found in DataFrame. Adding NaNs.
Column G_Prgm_Purp2 not found in DataFrame. Adding NaNs.
Column G_Prgm_Purp3 not found in DataFrame. Adding NaNs.
Column G_Prgm_Decide not found in DataFrame. Adding NaNs.
Column G_PrgmEffC_5y not found in DataFrame. Adding NaNs.
Column G_Prgm_2007NoHHB not found in DataFrame. Adding NaNs.
Appending data for column Survey_Round
Appending data for column Community
Appending

In [15]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'cid': 'CID',
    'program_id': 'G_Prgm_ID',
    'program_name': 'G_Prgm_Name',
    's2_q8': 'G_Prgm_M',
    's2_q9': 'G_PrgmM_FIntro',
    's2_q10': 'G_PrgmM_StillRun',
    's2_q11': 'G_PrgmM_NoHHB'
}


merged_df.rename(columns=rename_mapping, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('3. merged_S2P3_Governence 3.csv', index=False)

